### Raw Tensorflow Neural Network
- Simple implementation of neural network on tensorflow using mnist dataset

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [4]:
# load mnist dataset
train_data = pd.read_csv('data/mnist_train.csv')
test_data = pd.read_csv('data/mnist_test.csv')


In [5]:
X = train_data.drop(columns=['label'])
Y = train_data['label']
X.head()

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# split train and test data
scaler = MinMaxScaler()
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

#scale data to 0-1 range of values
x_train_scaled = scaler.fit_transform(x_train.astype(np.float64))
x_test_scaled = scaler.fit_transform(x_test.astype(np.float64))
#convert labels as int
y_train  = y_train.astype(int)
y_test  = y_test.astype(int)



In [7]:
def dense_to_one_hot(labels_dense, num_classes=10):
    """Convert class labels from scalars to one-hot vectors"""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot
    
# one hot encode y values
y_train = np.array(y_train)
y_train = dense_to_one_hot(y_train)

In [8]:
# create raw neural network

#define hyper-parameters
epochs = 500
learning_rate = 0.0001

#define network architecture
input_size = x_train.shape[1]
h1_size = 256
h2_size = 256
batch_size = 64
output_size = len(np.unique(np.array(Y)))
total_batch = int(len(x_train_scaled) / batch_size)

x = tf.placeholder("float", [None, input_size])
y = tf.placeholder("float", [None, output_size])
keep_prob = tf.placeholder("float", None) # dropout
#create weights and biases

#weights of hidden layers
w_h1= tf.Variable(tf.random.normal([input_size,h1_size]))
w_h2 = tf.Variable(tf.random.normal([h1_size,h2_size ]))
w_output = tf.Variable(tf.random.normal([h2_size,output_size]))
#biases of hidden layers
b_h1 = tf.Variable(tf.random.normal([h1_size]))
b_h2 = tf.Variable(tf.random.normal([h2_size]))
b_output = tf.Variable(tf.random.normal([output_size]))

#create layer forward function
def nn_layer(x_input,w,b,acti,drop=None):
    z = tf.add(tf.matmul(x_input,w),b) # linear function
    if acti == 'relu':
        a = tf.nn.relu(z)
    elif acti == 'sigmoid':
        a = tf.nn.sigmoid(z)
    elif acti == 'tanh':
        a = tf.nn.tanh(z)
    #perform dropout
    #a = tf.nn.dropout(a,drop)
    return a

# build neural network

a_1 = nn_layer(x,w_h1,b_h1,'relu') #layer activation 1
a_2 = nn_layer(a_1,w_h2,b_h2,'relu') #layer activation 2
output = nn_layer(a_2,w_output,b_output,'sigmoid') #layer activation output

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output,labels=y)) #compute loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) #create optimizer
train_opti = optimizer.minimize(loss)
#compute accuracy
correct_pred = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# train neural network
init = tf.global_variables_initializer() #initialize all tf variables 
sess = tf.Session()
sess.run(init)
for e in range(epochs):
    avg_cost = 0
    avg_acc = 0
    #train per batch data
    for i in range(1,total_batch+1):
        #create training data batches
        batch_start = i *batch_size
        batch_end = batch_start + batch_size
        if batch_end > x_train_scaled.shape[0]:
            batch_end = x_train_scaled.shape[0]
        batch_x,batch_y = x_train_scaled[batch_start:batch_end,:],y_train[batch_start:batch_end,:]
        
        #train neural net
        _,loss_val,acc = sess.run([train_opti,loss,accuracy], feed_dict={x: batch_x, y: batch_y})
            
        avg_cost += loss_val /total_batch
        avg_acc += acc /total_batch
    if (e+1) % 10 == 0:
        print(f"Epoch: {e+1} / Loss: {avg_cost} / Accuracy: {avg_acc}") 
    
sess.close()
    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch: 10 / Loss: 2.003438232330289 / Accuracy: 0.3120950838414636
Epoch: 20 / Loss: 1.8336946956995053 / Accuracy: 0.5286061356707311
Epoch: 30 / Loss: 1.7285399384251448 / Accuracy: 0.6286918826219509
Epoch: 40 / Loss: 1.696612792952759 / Accuracy: 0.6613471798780483
Epoch: 50 / Loss: 1.679138533225874 / Accuracy: 0.7014338795731713
Epoch: 60 / Loss: 1.6650271101332297 / Accuracy: 0.7100562118902446
Epoch: 70 / Loss: 1.6551846278513336 / Accuracy: 0.7327791539634161
Epoch: 80 / Loss: 1.6461573312195334 / Accuracy: 0.7540491615853672
Epoch: 90 / Loss: 1.640423050559148 / Accuracy: 0.7645769817073192
Epoch: 100 / Loss: 1.6342068225508806 / Accuracy: 0.7774390243902453
Epoch: 110 / Loss: 1.629229284822941 / Accuracy: 0.783560403